In [1]:
import polars as pl
import numpy as np
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, balanced_accuracy_score
from sklearn.model_selection import KFold

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
df_polars_raiz = pl.read_parquet('../dataset.parquet')

In [3]:
df_polars = df_polars_raiz.sample(fraction=0.01, seed=42)

In [4]:
import ipaddress

def ip_to_int(ip: str) -> int:
    try:
        return int(ipaddress.ip_address(ip))  # Funciona tanto para IPv4 quanto IPv6
    except ValueError:
        return None

In [5]:
#df_polars = df_polars.with_columns([
#    pl.col('id.resp_h').map_elements(ip_to_int).alias('id.resp_h'),
#    pl.col('id.orig_h').map_elements(ip_to_int).alias('id.orig_h')
#])

In [6]:
df_polars = df_polars.with_columns([
    pl.col('duration').fill_null(0),
    pl.col('orig_bytes').fill_null(0),
    pl.col('resp_bytes').fill_null(0)
])

In [7]:
lista_colunas = df_polars.columns
colunas_para_spearman = ['id.resp_p', 'history', 'conn_state', 'id.orig_p', 'orig_ip_bytes', 'label']             
#['detailed-label', 'id.resp_p', 'history', 'id.orig_h', 'conn_state', 'id.orig_p', 'orig_ip_bytes']
colunas_para_dropar = [col for col in lista_colunas if col not in colunas_para_spearman]
df_polars = df_polars.drop(colunas_para_dropar)

In [8]:
df_polars = df_polars.drop_nulls()

In [9]:
X = df_polars.drop('label')
y = df_polars['label']       

In [10]:
#X = X.to_numpy()
#X[:, 0] = np.array([ip_to_int(ip) for ip in X[:, 0]])

# Treinamento

In [11]:
results = []

In [12]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim=32):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 8),
            nn.ReLU(), 
        )
        self.decoder = nn.Sequential(
            nn.Linear(8, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim) 
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [13]:
def startTrain(X, y, hidden_dim=8, epochs=10, learning_rate=0.001, batch_size = 5000):
    kfold = KFold(n_splits=5, shuffle=True)
    results_fold = []

    for fold, (train_idx, test_idx) in enumerate(kfold.split(X)):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Aplicar MinMaxScaler APENAS aos dados de treino
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Filtrar apenas os dados benignos (classe 0) para treinar o Autoencoder
        X_train = X_train[y_train == 0]

        # Converter para tensores PyTorch
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

        # Criar DataLoaders
        batch_size = batch_size
        train_dataset = TensorDataset(X_train_tensor)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        # Configuração do dispositivo e modelo
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        input_dim = X_train.shape[1]
        model = Autoencoder(input_dim, hidden_dim).to(device)

        # Definição da função de perda e otimizador
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        # Treinamento
        epochs = epochs
        start_training = time.time()
        for epoch in range(epochs):
            model.train()
            epoch_loss = 0
            for data in train_loader:
                inputs = data[0].to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, inputs)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item() * inputs.size(0)

        end_training = time.time()

        # Avaliação do limiar baseado no conjunto de treino
        model.eval()
        reconstruction_errors_train = []
        with torch.no_grad():
            for data in train_loader:
                inputs = data[0].to(device)
                outputs = model(inputs)
                reconstruction_error = torch.mean((outputs - inputs) ** 2, dim=1)
                reconstruction_errors_train.extend(reconstruction_error.cpu().numpy())

        reconstruction_errors_train = np.array(reconstruction_errors_train)
        threshold = np.percentile(reconstruction_errors_train, 95)  # Limiar baseado no percentil 95

        # Avaliação no conjunto de teste
        reconstruction_errors_test = []
        y_pred = []
        with torch.no_grad():
            for data in test_loader:
                inputs, targets = data[0].to(device), data[1].cpu().numpy()
                outputs = model(inputs)
                reconstruction_error = torch.mean((outputs - inputs) ** 2, dim=1).cpu().numpy()
                reconstruction_errors_test.extend(reconstruction_error)
                y_pred.extend((reconstruction_error > threshold).astype(int))

        # Conversão para numpy arrays
        reconstruction_errors_test = np.array(reconstruction_errors_test)
        y_pred = np.array(y_pred)

        # Cálculo de métricas
        accuracy = accuracy_score(y_test, y_pred)
        results_fold.append(accuracy)

    return np.mean(results_fold)


In [14]:
def grid_search(X, y):
    param_grid = {
        'hidden_dim': [8, 16],
        'epochs': [10, 20, 25],
        'learning_rate': [0.001, 0.0001],
        'batch_size': [128, 256, 512, 5000]  # Adicionando o parâmetro batch_size
    }

    best_accuracy = 0
    best_params = {}

    for hidden_dim in param_grid['hidden_dim']:
        for epochs in param_grid['epochs']:
            for learning_rate in param_grid['learning_rate']:
                for batch_size in param_grid['batch_size']:  # Iterando também sobre o batch_size
                    accuracy = startTrain(X, y, hidden_dim=hidden_dim, 
                                          epochs=epochs, learning_rate=learning_rate, batch_size=batch_size)
                    print(f"hidden_dim={hidden_dim}, epochs={epochs}, learning_rate={learning_rate}, batch_size={batch_size} -> Accuracy: {accuracy}")

                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_params = {
                            'hidden_dim': hidden_dim,
                            'epochs': epochs,
                            'learning_rate': learning_rate,
                            'batch_size': batch_size
                        }

    print("\nMelhores Hiperparâmetros encontrados:", best_params)
    print(f"Melhor Acurácia Média: {best_accuracy}")
    return best_params


In [15]:
inicio = time.time()
best_params = grid_search(X, y)
fim = time.time()

hidden_dim=8, epochs=10, learning_rate=0.001, batch_size=128 -> Accuracy: 0.9807702389090565
hidden_dim=8, epochs=10, learning_rate=0.001, batch_size=256 -> Accuracy: 0.9812259051514257
hidden_dim=8, epochs=10, learning_rate=0.001, batch_size=512 -> Accuracy: 0.9522237613508484
hidden_dim=8, epochs=10, learning_rate=0.001, batch_size=5000 -> Accuracy: 0.5228888059386978
hidden_dim=8, epochs=10, learning_rate=0.0001, batch_size=128 -> Accuracy: 0.9082403045629786
hidden_dim=8, epochs=10, learning_rate=0.0001, batch_size=256 -> Accuracy: 0.8720662803328324
hidden_dim=8, epochs=10, learning_rate=0.0001, batch_size=512 -> Accuracy: 0.833244735664201
hidden_dim=8, epochs=10, learning_rate=0.0001, batch_size=5000 -> Accuracy: 0.3850612932527584
hidden_dim=8, epochs=20, learning_rate=0.001, batch_size=128 -> Accuracy: 0.9724367690629551
hidden_dim=8, epochs=20, learning_rate=0.001, batch_size=256 -> Accuracy: 0.9772239591844121
hidden_dim=8, epochs=20, learning_rate=0.001, batch_size=512 -> A

In [16]:
duracao = fim - inicio
print(duracao)

3570.3299725055695


In [17]:
print("\nMelhores Hiperparâmetros encontrados:", best_params)


Melhores Hiperparâmetros encontrados: {'hidden_dim': 16, 'epochs': 20, 'learning_rate': 0.001, 'batch_size': 512}
